# 0. Prep

In [ ]:
%pylab inline

In [ ]:
import os
import string
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

### Import Texts

In [ ]:
with open("Gold - Debates in DH.txt")  as fin:
    debates_text = fin.read()

In [ ]:
debates_nopunc = "".join([char for char in debates_text if char not in string.punctuation])
debates_tokens = debates_nopunc.lower().split()
debates_nostops = [token for token in debates_tokens if token not in ENGLISH_STOP_WORDS and len(token)>1]

# 1. Flyover Analysis

### Most Frequent Tokens

In [ ]:
c = Counter(debates_nostops)
c.most_common(10)

### Most Frequent Bi-grams

In [ ]:
debates_bigrams = [debates_nostops[i]+'_'+debates_nostops[i+1] for i in range(len(debates_nostops)-1)]
d = Counter(debates_bigrams)
d.most_common(10)

# 2. PCA

### Parameters

In [ ]:
top_n = 300
window = 3
model = 'exclude_ny' # options: 'all_tokens', 'exclude_ny', 'exclude_dh', 'exclude_ny_dh'

top_words = [x for x,y in c.most_common(top_n)]
top_set = set(top_words)
top_dict = {word:[] for word in top_words}

### Keyword-Context Collection

In [ ]:
for i in range(len(debates_nostops)):
    
    if model not in ['all_tokens', 'exclude_ny', 'exclude_dh', 'exclude_ny_dh']:
        
        print('NOT A VALID MODEL')
        break
        
    if debates_nostops[i] in top_set:
        for j in range(i-window,i+window+1):
            if j!=i and (i-j)>0 and j<len(debates_nostops) and debates_nostops[j] in top_set:
                if model == 'all_tokens':
                    
                    top_dict[debates_nostops[i]].append( debates_nostops[j] )
                    
                elif model == 'exclude_ny':
                    
                    if not (debates_nostops[j]=='york' and debates_nostops[j-1]=='new' ) and not (debates_nostops[j]=='new' and debates_nostops[j+1]=='york' ) :
                        top_dict[debates_nostops[i]].append( debates_nostops[j] )
                
                elif model == 'exclude_dh':

                    if not (debates_nostops[j][:5]=='human' and debates_nostops[j-1]=='digital' )\
                    and not (debates_nostops[j]=='digital' and debates_nostops[j+1][:5]=='human' ):
                        top_dict[debates_nostops[i]].append( debates_nostops[j] )
                        
                elif model == 'exclude_ny_dh':

                    if not (debates_nostops[j][:5]=='human' and debates_nostops[j-1]=='digital' )\
                    and not (debates_nostops[j]=='digital' and debates_nostops[j+1][:5]=='human' )\
                    and not (debates_nostops[j]=='york' and debates_nostops[j-1]=='new' )\
                    and not (debates_nostops[j]=='new' and debates_nostops[j+1]=='york' ) :
                        top_dict[debates_nostops[i]].append( debates_nostops[j] )
                
for key in top_dict.keys():
    top_dict[key] = dict(Counter(top_dict[key]))

### DTM, Normalize, PCA

In [ ]:
# DTM
dv = DictVectorizer()
dtm = dv.fit_transform( [top_dict[key] for key in top_dict.keys()] ).toarray()
feat_list = dv.get_feature_names()

# Laplace Smooth
row_sums = dtm.sum(axis=1)
dtm = (dtm + 1) / (row_sums[:, np.newaxis] + len(dtm))

# PCA

# Since sklearn's implementation of PCA is probabalistic, have found that
# learning more components than plan to use results in greater stability
pca = PCA(n_components=4)
pca.fit(dtm)

# Print variance explained by first two PCs
pca.explained_variance_ratio_[:2]

### Export Smoothed DTM

In [ ]:
# e.g. for viz in R
pd.DataFrame(dtm, columns=feat_list, index=top_dict.keys()).to_csv('dtm_smooth.csv')

# 3. Visualize

### Biplot

In [ ]:
# code adapted from: https://github.com/teddyroland/python-biplot

from matplotlib import pyplot as plt

xvector = pca.components_[0] # see 'prcomp(my_data)$rotation' in R
yvector = pca.components_[1]

xs = pca.transform(dtm)[:,0] # see 'prcomp(my_data)$x' in R
ys = pca.transform(dtm)[:,1]

plt.figure(figsize=(10,10))

for i in range(len(xs)):
# circles project documents (ie rows from dtm) as points onto PC axes
    plt.plot(xs[i], ys[i], 'bo')
    plt.text(xs[i]*1.02, ys[i]*1.02, list(top_dict.keys())[i], color='b')

for i in range(len(xvector)):
# arrows project features (ie columns from dtm) as vectors onto PC axes
    plt.arrow(0, 0, xvector[i]*max(xs), yvector[i]*max(ys),
              color='r', width=0.000005, head_width=0.000025)
    plt.text(xvector[i]*max(xs)*1.02, yvector[i]*max(ys)*1.02,
            feat_list[i], color='r')

    
plt.show()

# 4. Loading Analysis

### DataFrame: Magnitude, Dot Product with 'digital'/'humanities'

In [ ]:
loading_df = pd.DataFrame(pca.components_[:2].T, columns = ['LOADING_1','LOADING_2'], index=feat_list)
loading_df['MAGNITUDE'] = abs(loading_df['LOADING_1']) + abs(loading_df['LOADING_2'])

pc_norm_l2 = pca.components_[:2].T/np.linalg.norm(pca.components_[:2].T, axis=1)[:, np.newaxis]

d_dex = loading_df.index.tolist().index('digital')
loading_df['DOT_DIGITAL'] = np.dot(pc_norm_l2[d_dex], pc_norm_l2.T)

h_dex = loading_df.index.tolist().index('humanities')
loading_df['DOT_HUMANITIES'] = np.dot(pc_norm_l2[h_dex], pc_norm_l2.T)

### Export DataFrame

In [ ]:
loading_df.to_csv(model+'_loadings.csv')